In [ ]:
import re
from typing import List, Any
from langchain.schema.messages import HumanMessage, AIMessage
from langchain.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

# 1. 임베딩 모델 & 벡터 DB 로드
embeddings_model = OllamaEmbeddings(model="bge-m3:latest")
menu_db = FAISS.load_local(
    "./db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

# 2. 문의 분류 함수
def classify_intent(user_message: str) -> str:
    if any(x in user_message for x in ["추천", "뭐 먹을까", "인기", "추천해"]):
        return "recommend"
    if any(x in user_message for x in ["얼마", "가격", "비용", "비싸", "저렴"]):
        return "price"
    if any(x in user_message for x in ["메뉴", "종류", "있는지", "파나요", "있나요"]):
        return "menu"
    return "menu"

# 3. 메뉴 정보 추출
def extract_menu_info(doc) -> dict:
    content = doc.page_content
    menu_name = doc.metadata.get('menu_name', 'Unknown') if hasattr(doc, "metadata") else "Unknown"
    price_match = re.search(r'₩[\d,]+', content)
    description_match = re.search(r'설명:\s*(.+?)(?:\n|$)', content, re.DOTALL)
    return {
        "name": menu_name,
        "price": price_match.group(0) if price_match else "가격 정보 없음",
        "description": description_match.group(1).strip() if description_match else "설명 없음"
    }

# 4. 답변 생성 함수들
def answer_menu_query(messages: List[Any], user_message: str) -> str:
    docs = menu_db.similarity_search(user_message, k=4)
    if not docs:
        return "죄송합니다. 해당 메뉴를 찾을 수 없습니다."
    infos = [extract_menu_info(doc) for doc in docs]
    reply = "\n".join([f"- {info['name']}: {info['description']} ({info['price']})" for info in infos])
    return f"저희 카페의 관련 메뉴입니다:\n{reply}"

def answer_price_query(messages: List[Any], user_message: str) -> str:
    docs = menu_db.similarity_search("메뉴 가격", k=5)
    infos = [extract_menu_info(doc) for doc in docs]
    reply = "\n".join([f"- {info['name']}: {info['price']}" for info in infos])
    return f"주요 메뉴의 가격은 다음과 같습니다:\n{reply}"

def answer_recommend_query(messages: List[Any], user_message: str) -> str:
    docs = menu_db.similarity_search(user_message, k=3)
    if not docs:
        docs = menu_db.similarity_search("인기 메뉴", k=3)
    infos = [extract_menu_info(doc) for doc in docs]
    reply = "\n".join([f"- {info['name']}: {info['description']} ({info['price']})" for info in infos])
    return f"추천드리는 메뉴입니다:\n{reply}"

# 5. 간단한 대화 에이전트
def create_react_agent():
    def agent_step(messages: List[Any]) -> List[Any]:
        user_message = messages[-1].content if messages else ""
        intent = classify_intent(user_message)
        if intent == "recommend":
            answer = answer_recommend_query(messages, user_message)
        elif intent == "price":
            answer = answer_price_query(messages, user_message)
        else:
            answer = answer_menu_query(messages, user_message)
        messages.append(AIMessage(content=answer))
        return messages
    return agent_step

# 6. 실행 예시
if __name__ == "__main__":
    messages = []
    agent = create_react_agent()
    print("카페 챗봇에 오신 것을 환영합니다! (종료: exit 또는 quit)")
    while True:
        user_input = input("손님: ")
        if user_input.strip().lower() in ["exit", "quit"]:
            break
        messages.append(HumanMessage(content=user_input))
        messages = agent(messages)
        print("AI:", messages[-1].content)



C:\Users\user\AppData\Local\Temp\ipykernel_24872\151400800.py:8: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings_model = OllamaEmbeddings(model="bge-m3:latest")


카페 챗봇에 오신 것을 환영합니다! (종료: exit 또는 quit)
AI: 저희 카페의 관련 메뉴입니다:
- 아이스 아메리카노: 진한 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 아이스 커피입니다. 깔끔하고 시원한 맛이 특징이며, 원두 본연의 풍미를 느낄 수 있습니다. 더운 날씨에 인기가 높습니다. (₩4,500)
- 콜드브루: 찬물에 12-24시간 우려낸 콜드브루 원액을 사용한 시원한 커피입니다. 부드럽고 달콤한 맛이 특징이며, 산미가 적어 누구나 부담 없이 즐길 수 있습니다. 얼음과 함께 시원하게 제공됩니다. (₩5,000)
- 프라푸치노: 에스프레소와 우유, 얼음을 블렌더에 갈아 만든 시원한 음료입니다. 부드럽고 크리미한 질감이 특징이며, 휘핑크림을 올려 달콤함을 더했습니다. 여름철 인기 메뉴입니다. (₩7,000)
- 아메리카노: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다. (₩4,500)
AI: 추천드리는 메뉴입니다:
- 바닐라 라떼: 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴입니다. 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러지며, 휘핑크림 토핑으로 더욱 풍성한 맛을 즐길 수 있습니다. (₩6,000)
- 티라미수: 이탈리아 전통 디저트로 마스카포네 치즈와 에스프레소에 적신 레이디핑거를 층층이 쌓아 만들었습니다. 부드럽고 달콤한 맛이 특징이며, 코코아 파우더로 마무리하여 깊은 풍미를 자랑합니다. (₩7,500)
- 카페라떼: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 질감과 부드러운 맛이 특징이며, 다양한 시럽과 토핑 추가가 가능합니다. 라떼 아트로 시각적 즐거움도 제공합니다. (₩5,500)
